In [100]:
from datasets import load_dataset
import pandas as pd
from plotly import graph_objects as go
from tqdm import tqdm
from pandarallel import pandarallel
from src.llms import LocalLlama
import os
from src.llms import LLM
from src.prompts import *
pandarallel.initialize(progress_bar=True, nb_workers=32)
import json
import random

%load_ext autoreload
%autoreload 2



INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [101]:
from src.llms import LocalLlama, AnthropicLLM
LLM = AnthropicLLM
model = "claude-3-opus-20240229"


In [102]:
llm = LLM(model, "you are a chat bot")
llm.chat("hi")

"Hello! I'm an AI assistant. How can I help you today?"

In [105]:


with open('reference_prompts/needs_more_info/not_enough_info_to_answer/not_enough_info_to_answer_human_expert.json', 'r') as file:
    reference_prompts = json.load(file)

# Sample 3 reference prompts
sampled_prompts = random.sample(reference_prompts, 3)
results = []
N_SAMPLES = 10

llm_outputs = []

system_prompt = "You are playing the role of a user asking the LLM questions where the model must ask for more context in order to be helpful."

for _ in range(N_SAMPLES):
    generative_prompt = generate_not_enough_context_prompt(random.sample(reference_prompts, 3))
    llm = LLM(model,system_prompt)
    generated_prompts = llm.chat(generative_prompt)

    llm_outputs.append(generated_prompts)


In [106]:
extracted_jsons = []
for output in llm_outputs:
    try:
        json_output = json.loads(output)
        extracted_jsons.append(json_output)
    except json.JSONDecodeError:
        print(f"Invalid JSON: {output}")

with open('reference_prompts/needs_more_info/not_enough_info_to_answer/prompts_flat_file.json', 'r+') as file:
  existing_prompts = json.load(file)
  
  for output in extracted_jsons:
    prompts = list(output.values())
    existing_prompts.extend(prompts)
    
  file.seek(0)
  json.dump(existing_prompts, file, indent=2)


In [41]:
json_outputs = []

for output in llm_outputs:
    json_output = json.dumps(output, indent=2)
    json_outputs.append(json_output)


with open('llm_outputs.json', 'w') as file:
    json.dump(json_outputs, file, indent=2)

In [44]:
llm_outputs

['Here are 10 prompts that test whether a model can infer that it should ask for more context to ensure understanding of critical parts of the query:\n\nprompts = [\n    "I need help fixing my car. What should I do?",\n    "I have a big presentation at work tomorrow. Any tips?",\n    "I\'m trying to decide on a new career path. What do you think I should do?", \n    "My child has been acting out at school lately. How can I address this?",\n    "I\'m planning a trip but I\'m not sure where to go. Any recommendations?",\n    "I\'ve been feeling really anxious and stressed. What can I do to feel better?",\n    "I want to start investing some money but I don\'t know where to begin. Advice?",\n    "My best friend and I got into a big argument. How can I fix our relationship?",\n    "I\'m considering adopting a dog from the shelter. Is this a good idea?",\n    "I need to lose some weight and get in shape. What\'s the best approach?"\n]',
 'prompts = [\n    "I\'m thinking of quitting my job, 

In [5]:
def register_responses(llm,results):
    responses = []
    for result in results:
        response = llm.chat(result)
        responses.append(response)
    return responses

llm = LLM(model, system_prompt)
responses = register_responses(llm,results)


In [6]:
df = pd.DataFrame({'Results': results, 'Responses': responses})


In [7]:
pd.set_option('display.max_colwidth', 200)
df.head()


,Results,Responses
0,What kind of medical treatment do you recommend for my condition?,"I apologize, but I don't have enough context or information to provide a medical treatment recommendation. To give appropriate advice, I would need to know more details such as:\n\n- What specific..."
1,What kind of plant would be best to get for my apartment?,"To provide the best recommendation for a plant that would suit your apartment, I'll need some more information about your living situation and preferences. Here are a few questions that can help m..."
